# **Minggu 5: Feature Detection**


(*Tutorial Rujukan: [Dokumentasi OpenCV untuk Feature Detection](https://docs.opencv.org/3.4/db/d27/tutorial_py_table_of_contents_feature2d.html)*)

## Review: Ekstraksi objek pada citra

Pada saat melihat sebuah foto atau gambar digital, manusia dapat dengan mudah membedakan objek yang digambarkan pada foto tersebut. Sebagai contoh, dari gambar berikut:

![alt text](https://drive.google.com/uc?id=1kc8QA4GIHqaQusFlcGd91cUNnb3nr7GU)

Manusia dapat dengan mudah membedakan jeruk dan daun, jeruk yang dekat dengan yang jauh, serta menghitung jumlah jeruk yang ada, misalnya.

Untuk komputer, hal ini menjadi kompleks karena komputer hanya melihat serangkaian pixel, tanpa kemampuan untuk memisahkan fitur tersebut. 

Pada [pertemuan sebelumnya](https://colab.research.google.com/drive/1To2FZnLbxdhRSPW3lZlUQymaV2T5lF_m), telah dibahas mengenai cara 'mengajarkan' komputer agar mampu melihat dan membedakan warna (Hue) sehingga dapat digunakan untuk ekstraksi fitur. Pada [minggu selanjutnya](https://colab.research.google.com/drive/1FpvJQnGb3_A85oktAXJOzGYG24DK26u1) juga telah dibahas mengenai metode filtering dan thresholding yang dapat digunakan untuk menonjolkan **tepi** sebuah objek sehingga dapat dilakukan ekstraksi fitur, misalnya delineasi garis pantai.

![alt text](https://slideplayer.com/slide/14894726/91/images/6/Images+as+functions%E2%80%A6+Edges+look+like+steep+cliffs.jpg)

Dengan menggunakan nilai piksel pada sebuah citra digital, kita dapat memberikan informasi pada komputer agar dapat mengenali suatu objek. Namun bagaimana dengan kedalaman objek tersebut? Pembahasan minggu ini merupakan pengantar untuk materi *depth reconstruction* melalui pengolahan citra digital.





## Apa itu Fitur?

'Fitur' merupakan bagian dari citra yang dapat diidentifikasi dengan mudah oleh komputer. Sebagai contoh, pada gambar berikut: 

![alt text](https://drive.google.com/uc?id=1lDujYsNKdilG_d-Uv4gt0hzoPzBKDZ6q)

Manakah yang merupakan 'fitur' yang mudah untuk diidentifikasi?

Jawabnya adalah ujung persegi panjang yang ditandai oleh kotak berwarna merah. Kedua fitur lain cukup sulit untuk diidentifikasi:
*   Kotak warna biru dapat berada pada posisi manapun di dalam persegi panjang warna hijau
*   kotak warna hitam dapat berada dimana saja selama berada pada tepian persegi panjang.

Dengan demikian, sebuah citra dapat dilatih untuk mencari titik-titik yang mudah dilacak (*Good Features To Track*) dengan membuat kategori untuk tiap piksel dengan tetangga piksel masing-masing. 

Terdapat beberapa algoritma untuk melakukan deteksi pojok ('Corner Detector') dari sebuah citra digital. Beberapa yang cukup populer adalah:

*   [Harris Corner Detector](https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html)
*   [Shi-Tomasi dan Good Features to Track](https://docs.opencv.org/3.4/d4/d8c/tutorial_py_shi_tomasi.html)

Lakukan latihan berikut untuk memahami mengenai Good Features to Track pada sebuah Citra Digital






### Latihan 1: Menggunakan Corner Detector

Lakukan latihan berikut menggunakan kedua metode yang tersedia pada OpenCV (Harris Corner dan Shi-Tomasi). Gunakan gambar yang berbeda dan bandingkan jumlah corner yang berhasil dideteksi oleh kedua algoritma tersebut.

In [1]:
# Menggunakan Shi-Tomasi GFTT untuk deteksi ujung (corner detection)

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

# gunakan gambar yang disediakan di eLOK
img = cv.imread('gedungpusat.jpg')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY) 


# deteksi pojok dengan GFTT
corners = cv.goodFeaturesToTrack(gray,1000,0.01,10)
corners = np.int0(corners)

# menampilkan jumlah titik terdeteksi dengan fungsi numpy (np.ndarray.shape)
print(corners.shape)

# untuk ditampilkan di Matplotlib, urutan band dibalik
rgb = cv.cvtColor(img,cv.COLOR_BGR2RGB)

# perbesar ukuran hasil plotting 
plt.rcParams["figure.figsize"] = (20,20)

# untuk tiap pojok yang terdeteksi, munculkan pada gambar
for i in corners:
    x,y = i.ravel()
    cv.circle(rgb,(x,y),3,255,-1)
plt.imshow(rgb),plt.show()

error: ignored

In [2]:
# Mendownload gambar Lenna
!wget https://pns2019.github.io/images/Lenna.png
import cv2
import numpy as np

img = cv2.imread('lena.png',0)
#gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

kaze = cv2.KAZE_create()
kp = kaze.detect(img)

kps = sorted(kp, key=lambda x: -x.response)[:32]
# computing descriptors vector
kps, dsc = kaze.compute(img, kp)

print(kps)

--2020-08-31 04:36:51--  https://pns2019.github.io/images/Lenna.png
Resolving pns2019.github.io (pns2019.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to pns2019.github.io (pns2019.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473831 (463K) [image/png]
Saving to: ‘Lenna.png’

Lenna.png           100%[===================>] 462.73K  --.-KB/s    in 0.08s   

2020-08-31 04:36:51 (5.60 MB/s) - ‘Lenna.png’ saved [473831/473831]

[]


## Deteksi Keypoints (Feature Detection)

*(Rujukan: http://www.fossreview.com/2018/06/studying-digital-image-with-opencv4.html)*


---


Deteksi ujung (corner detection) dapat digunakan untuk 
mengidentifikasi fitur pada satu foto dengan baik. Namun demikian, metode ini kurang dapat digunakan dengan baik apabila diperlukan untuk mengidentifikasi titik yang sama pada foto yang berbeda. Kebutuhan ini akan kita jumpai pada saat kita mencari hubungan geometri pada dua buah foto, misalnya (ingat materi [minggu ke-4](https://colab.research.google.com/drive/1ZmddpA9v5dnzKJ8Up_So0AfjKjroOojZ)).

Sebagai contoh, pada gambar berikut:

![alt text](https://drive.google.com/uc?id=1c3g2BTFgSunu2BXqKLViVkr6u3pjT8Of)

Corner detection tidak akan banyak berguna untuk mencari titik-titik yang sama dan berpasangan pada kedua buah gambar (bayangkan '*tie-points*' pada fotogrametri). Hal ini disebabkan karena gambar kedua memiliki posisi yang berbeda (mengalami rotasi, translasi dan/atau skala), sehingga deteksi ujung tidak dapat digunakan. 

Gambar berikut menunjukkan kelemahan corner detector apabila digunakan pada gambar dengan perubahan skala (misalnya beda tinggi terbang foto udara)

![alt text](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/sift_scale_invariant.jpg)

Pada pelajaran minggu sebelumnya telah diberikan contoh bahwa dari beberapa **titik sekutu** pada dua buah foto dapat dicari matriks, atau **HOMOGRAPHY** yang menyatakan hubungan antara kedua sistem foto tersebut. Dengan demikian, permasalahannya adalah bagaimana mencari titik-titik yang dapat dideteksi pada kedua buah foto sekaligus dapat dihubungkan antara titik satu dengan yang lain?

Untuk itu, dikembangkan suatu metode untuk melakukan deteksi *keypoint* pada tiap foto. **Keypoint** dapat disebut sebagai sebuah 'basisdata' untuk tiap titik dengan karakteristik tertentu berdasarkan atas nilai piksel tetangganya. Dengan membentuk basisdata titik yang cukup detil, dua buah titik dapat saling dipasangkan apabila memiliki kemiripan yang cukup. Dengan demikian, matriks homography dapat dihitung dan rekonstruksi posisi foto dapat dilakukan sebagaimana materi yang telah disampaikan sebelumnya.

Pada OpenCV, terdapat beberapa fungsi deteksi keypoints, antara lain:

*   [SIFT (Scale-Invariant Feature Transform)](https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html). Merupakan metode yang paling terkenal dan banyak digunakan
*   [SURF (Speeded-Up Robust Feature)](https://docs.opencv.org/3.4/df/dd2/tutorial_py_surf_intro.html).
* [FAST](https://docs.opencv.org/3.4/df/d0c/tutorial_py_fast.html) 
* [BRIEF (Binary Robust Independent Elementary Features)](https://docs.opencv.org/3.4/dc/d7d/tutorial_py_brief.html). Merupakan descriptor yang digunakan untuk menghitung hasil deteksi secara lebih efisien. BRIEF seringkali dipasangkan dengan FAST.
* [ORB (Oriented FAST and Rotated BRIEF)](https://docs.opencv.org/3.4/d1/d89/tutorial_py_orb.html), merupakan alternatif gratis untuk SIFT dan SURF (keduanya memiliki paten)
* [A-KAZE](https://docs.opencv.org/3.4/db/d70/tutorial_akaze_matching.html). Salah satu feature detector terbaru yang banyak digunakan di software SfM Opensource


Fungsi-fungsi di atas dapat dipanggil dengan menggunakan fungsi features2d pada OpenCV sebagai berikut:

```
sift = cv2.xfeatures2d.SIFT_create()
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(nfeatures=1500)

# FAST dan BRIEF
# FAST detector
star = cv.xfeatures2d.StarDetector_create()
# BRIEF extractor
brief = cv.xfeatures2d.BriefDescriptorExtractor_create()
```

Karena algoritma SIFT dan SURF dipatenkan dan berbayar sedangkan OpenCV adalah perangkat lunak opensource, mulai OpenCV versi 4 semua fungsi tersebut dipisahkan dari modul utama OpenCV. Dengan demikian, untuk menggunakan fungsi tersebut dapat digunakan OpenCV versi lama (Versi 3) yang masih mengandung modul SIFT dan SURF.

### Latihan 2: Menggunakan ORB dan KAZE

ORB dan KAZE merupakan contoh feature detector yang bersifat opensource, sehingga dapat digunakan secara gratis pada OpenCV. 

Lakukan latihan berikut dengan gambar yang berbeda. Apabila script menggunakan gambar grayscale, tampilkan gambar berwarna dengan menggunakan Matplotlib.

In [3]:
# Latihan ORB

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

# membaca citra
img = cv.imread('webarebears.jpg',0)

# membuat ORB detector
orb = cv.ORB_create()

# kp: variabel untuk menyimpan keypoint yang berhasil dideteksi
kp = orb.detect(img,None)

# menghitung deskriptor
kp, des = orb.compute(img, kp)

# berapa jumlah titik terdeteksi?
#print(len(kp))

# menggambar keypoint yang berhasil diidentifikasi 
img2 = cv.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
plt.imshow(img2), plt.show()


error: ignored

### Latihan 3: Menggunakan SIFT dan SURF

Untuk menggunakan modul SIFT dan SURF, terlebih dahulu uninstall OpenCV bawaan 

In [ ]:
# untuk menjalankan fungsi detektor SIFT, SURF dan ORB pada OpenCV,
# terlebih dahulu uninstall OpenCV versi 4 yang secara default terinstall
# pada Google Colab, kemudian lakukan kembali instalasi versi OpenCV 
# yang lebih rendah sehingga fungsi SIFT dkk dapat digunakan

!pip uninstall opencv-python -y
# downgrade OpenCV a bit since some none-free features are not avilable
!pip install opencv-contrib-python==3.4.2.17 --force-reinstall


In [ ]:
# Setelah OpenCV versi 3 berhasil diinstal, barulah fungsi berikut dapat dijalankan

import cv2
from matplotlib import pyplot as plt

img = cv2.imread('webarebears.jpg')
# konversi ke warna abu2 agar menjadi satu band
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# menghitung fitur dengan SIFT
sift = cv2.xfeatures2d.SIFT_create()
kp = sift.detect(gray,None)

# berapa jumlah titik terdeteksi?
#print(len(kp))

# menggambar keypoint yang berhasil diidentifikasi 
img3 =cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(img3), plt.show()

## Feature Matching

Rujukan: https://pysource.com/2018/03/23/feature-matching-brute-force-opencv-3-4-with-python-3-tutorial-26/


Apabila feature detector digunakan untuk mencari titik yang dapat dipasangkan, maka **Feature Matching** digunakan untuk memasangkan masing-masing keypoint yang sudah terdeteksi pada satu citra dengan titik titik yang memiliki kesamaan karakter pada citra lain. Dengan demikian, dapat dikatakan bahwa hasil dari feature matching adalah **Tie-Points** pada sebuah proses fotogrametri.

![alt text](https://docs.opencv.org/3.4/matcher_flann.jpg)

Pada Gambar di atas, lingkaran berwarna merah merupakan fitur (keypoints) yang dideteksi dengan menggunakan *Feature Detector* pada OpenCV, sedangkan garis-garis berwarna hijau merupakan titik-titik yang berhasil dipasangkan berdasarkan atas kesamaan karakteristik (*feature matching*).

Terdapat beberapa algoritma yang berbeda pula pada metode Feature Matching, antara lain:


*   Brute-Force Matching
*   FLANN

(Panduan untuk keduanya dapat dilihat [di sini](https://docs.opencv.org/3.4/dc/dc3/tutorial_py_matcher.html))

Penggunaan kedua model feature matching tersebut perlu disesuaikan dengan algoritma keypoint detector yang digunakan.



### Latihan 4: Feature Detection and Matching

In [ ]:
# Contoh Script untuk feature detection and Matching
# Modifikasi script ini untuk mencoba metode yang berbeda 

import numpy as np
import cv2
from matplotlib import pyplot as plt

# Gunakan gambar dari eLOK
img1 = cv2.imread('webarebears.jpg')          # gambar yang dituju
img2 = cv2.imread('ice_bear.jpg')             # gambar yang dicari
gray1= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray2= cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

# Menggunakan Detector SIFT
sift = cv2.xfeatures2d.SIFT_create()

# Mencari Keypoint dengan SIFT
kp1, des1 = sift.detectAndCompute(gray1,None)
kp2, des2 = sift.detectAndCompute(gray2,None)

# Melakukan Matching dari hasil deteksi keypoints menggunakan
# BruteForce Matcher
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Uji rasio matching sederhana
good = []
for m,n in matches:
    if m.distance < 0.5*n.distance:
        good.append([m])
img3 = None

# menggambar hasil match pada gambar baru (IMG3)
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,img3,flags=2)
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)),plt.show()

## Resume

Feature detector dan Feature Matching merupakan metode untuk 1) mencari dan membuat basisdata titik untuk dipasangkan, dan 2) memasangkan tiap titik berdasarkan kesamaan karakteristik tertentu. Dalam fotogrametri, kedua metode ini berguna untuk mencari **tie points** secara otomatis pada dua buah gambar stereo. Selanjutnya, setelah titik-titik dapat diidentifikasi dan dipasangkan, dapat direkonstruksi geometri pengambilan gambar kedua foto tersebut meskipun *tanpa menggunakan GCP atau posisi awal kamera*. Hal ini akan dibahas lebih lanjut pada pertemuan minggu depan mengenai SfM (Structure from Motion).

Rangkuman algoritma yang tersedia pada OpenCV dapat dilihat pada gambar berikut:

![alt text](https://drive.google.com/uc?id=1DnWKbvF5rOqXnRQhIn6Qp-aqcjsni_ik)

### Latihan Soal

1.   Gunakan gambar bebas (bisa foto/cari di internet) dan lakukan **deteksi corner** menggunakan 1) metode Harris dan 2) GFTT. Bandingkan hasil kedua metode tersebut. (Latihan 1)
Mana yang lebih baik menurut anda? 
Mengapa? 
Berapa jumlah corner yang berhasil dideteksi oleh masing-masing metode?
2.   Gunakan gambar bebas (bisa foto/cari di internet) dan lakukan **deteksi keypoint** menggunakan: SIFT, SURF, BRIEF dan ORB. (Latihan 2 dan 3)
Mana yang lebih baik menurut anda? 
Mengapa? 
Berapa jumlah keypoint yang berhasil dideteksi oleh masing-masing metode?
3.   Gunakan gambar **wajah masing-masing** (bisa dengan foto/selfie) untuk mendemokan fungsi Feature Detection and Matching pada OpenCV. (Latihan 4) Gunakan masing-masing fungsi SIFT, SURF dan ORB untuk melakukan deteksi. Bandingkan jumlah keypoint yang terdeteksi berikut akurasi dari masing-masing metode.


